# Link based ranking



## 📚 Exercise 1: Page rank  (Eigen-vector method)

In the first exercise, we implement the Page Rank algorithm using Eigen vector method.


### Goal:
1. Implementing Page rank with Eigen-vector method.
2. Testing the implemented method on a small Web with three pages (with four different link structures).

### What you are learning in this exercise:
1. Analyzing the output of PageRank for different link structures.

**Preliminaries:** If you want to normalize a vector to L1-norm or L2-norm, you can use `numpy` library as follows:

In [1]:
from __future__ import print_function, division
import numpy as np

pr = np.array([1,2,3])
print("L1-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,1)))
print("L2-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,2)))

L1-norm of [1 2 3] is [0.16666667 0.33333333 0.5       ]
L2-norm of [1 2 3] is [0.26726124 0.53452248 0.80178373]


## Links structure
Consider a tiny Web with three pages A, B and C with no inlinks, and with initial PageRank = 1. Initially, none of the pages link to any other pages and none link to them. 
Run your algorithm for the following cases, and calculate the PageRank for
each case:

1. Link page A to page B.
2. Link all pages to each other.
3. Link page A to both B and C, and link pages B and C to A.
4. Use the previous links and add a link from page C to page B.

**Hints:**
Please note that for this section, we are using the theoretical PageRank computation (without source of rank). See the slide "Transition Matrix for Random Walker" in the lecture notes. Columns of link matrix are from-vertex, rows of link matrix are to-vertex. We take the eigenvector with the largest eigenvalue.
We only care about final ranking of the probability vector. You can choose the normalization (or not) of your choice.

In [2]:
def create_Rmatrix(L):
    X = np.sum(L,axis=0)
    n_nodes = L.shape[0]
    R = np.zeros((n_nodes, n_nodes))
    for i in range(L.shape[0]):
        for j in range(L.shape[1]):
            R[i,j] = L[i,j] / X[0,j] if X[0,j] != 0 else 0
    return R

In [3]:
# Implement your code here

def pagerank_eigen(L, R=None):
#   Construct transition probability matrix from L
    if R is None: R = create_Rmatrix(L)
#     Compute eigen-vectors and eigen-values of R
    eigenvalues, eigenvectors = np.linalg.eig(R)
#     Take the eigen-vector with maximum eigven-value
    p = np.absolute(eigenvectors[:,np.argmax(np.absolute(eigenvalues))])
    return (R,p)

In [4]:
# Test with a sample case, e.g.
L = np.matrix([
    [0,1,1], 
    [1,0,1], 
    [1,1,0]
])
R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

L=[[0 1 1]
 [1 0 1]
 [1 1 0]]
R=[[0.  0.5 0.5]
 [0.5 0.  0.5]
 [0.5 0.5 0. ]]
p=[0.57735027 0.57735027 0.57735027]


## 📚 Exercise 2: Page rank  (Iterative method)

In the second exercise, we implement the Page Rank algorithm with an iterative approach.


### Goal:
1. Implementing Page rank with the iterative method.
2. Reading the `ca-GrQc.txt` dataset and constructing its link matrix
3. Testing the implemented algorithm on the constructed link matrix
4. comparing the run-time of iterative method and eigen-vector method.

### What you are learning in this exercise:
1. Benefits of the iterative approach over eigen-vector method.

The eigen-vector method has some numerical issues (when computing eigen-vector) and not scalable with large datasets.

We will apply the iterative method in the slide "Practical Computation of PageRank" of the lecture.

Dataset for practice: https://snap.stanford.edu/data/ca-GrQc.html. It is available within the same paths of this notebook.

In [5]:

def pagerank_iterative(L, R=None):
    if R is None: #We might want to compute R outside this function to avoid recomputing large matrix
        R = np.multiply(L, 1 / np.sum(L,axis=0))
        
    N = R.shape[0]
    e = np.ones(shape=(N,1))
    q = 0.9

    p = e
    delta = 1
    epsilon = 0.001
    i = 0
    while delta > epsilon:
        p_prev = p
        p = np.matmul(q * R, p_prev)
        p = p + (1-q) / N * e
        delta = np.linalg.norm(p-p_prev,1)
        i += 1

    print("Converged after {0} iterations".format(i))
    return R,p

### Construct link matrix from dataset

In [16]:
# Construct link matrix from file
n_nodes = 0 # number of nodes in the dataset
nodes_idx = dict() #Since the nodeIDs are not from 0 to N we need to build an index of nodes
nodes = [] #We also want to store nodeIDs to return the result of ranking vector

# Read the nodes
with open("ca-GrQc.txt") as f:
    for line in f:
        if '#' not in line:
            source = int(line.split()[0])
            target = int(line.split()[1])
            if source not in nodes_idx.keys():
                nodes_idx[source] = n_nodes
                nodes.append(source)
                n_nodes += 1
            if target not in nodes_idx.keys():
                nodes_idx[target] = n_nodes
                nodes.append(target)
                n_nodes += 1
print(n_nodes)
print(nodes[:3])

L = np.zeros((n_nodes, n_nodes))
# Read the edges
with open("ca-GrQc.txt") as f:
    for line in f:
        if "#" not in line:
            source = int(line.split()[0])
            target = int(line.split()[1])
            L[nodes_idx[target], nodes_idx[source]] = 1 #Columns of link matrix are from-vertices
print(L)

5242
[3466, 937, 5233]
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]


## Comparing the run-time of eigen-vector and iterative methods

### You will see that eigen-vector method is slow and has some numerical issues


#### First we pre-compute the R matrix and pass it as argument to Page rank algorithms.

In [7]:
R = np.multiply(L, 1 / np.sum(L,axis=0))  # note that this doesn't handle zero columns
print(R)

[[0.  0.5 0.5]
 [0.5 0.  0.5]
 [0.5 0.5 0. ]]


In [8]:
import time
start_time = time.time()
# Run PageRank
R,p = pagerank_eigen(L, R)
print("--- %s seconds ---" % (time.time() - start_time))
print("Ranking vector: p={0}".format(p))

--- 0.0009992122650146484 seconds ---
Ranking vector: p=[[0.57735027]
 [0.57735027]
 [0.57735027]]


### Iterative method

In [9]:
start_time = time.time()
# Run PageRank
R, p = pagerank_iterative(L, R)
print("--- %s seconds ---" % (time.time() - start_time))
print("Ranking vector: p={0}".format(p[:,0]))

Converged after 52 iterations
--- 0.010723114013671875 seconds ---
Ranking vector: p=[[0.33611637]
 [0.33611637]
 [0.33611637]]


## Now let's extract top-k nodes from the ranking vector

In [10]:
arr = np.array(p[:,0])
k = 3
k_idx = arr.argsort()[-k:][::-1]
print("Top-{0} nodes: {1}".format(k, np.array(nodes)[k_idx]))
print("Their scores: {0}".format(arr[k_idx]))

Top-3 nodes: [[3466]
 [3466]
 [3466]]
Their scores: [[[0.33611637]]

 [[0.33611637]]

 [[0.33611637]]]


## 📚 Exercise 3: Ranking Methodology  (Hard)
This exercise is theoretical and you don't need to write code for it:

1. Give a directed graph, as small as possible, satisfying all the properties mentioned below:

    1. There exists a path from node i to node j for all nodes i,j in the directed
graph. Recall, with this property the jump to an arbitrary node in PageRank
is not required, so that you can set q = 1 (refer lecture slides).

    2. HITS authority ranking and PageRank ranking of the graph nodes are different.

2. Give intuition/methodology on how you constructed such a directed graph with
the properties described in (a).

3. Are there specific graph structures with arbitrarily large instances where PageRank
ranking and HITS authority ranking are the same?

1. directed graph with: path from node i to j for all nodes i,j; hits authority ranking and pagerank ranking of the graph nodes are different

voir correction

## 📚 Exercise 4: Hub and Authority

In the last exercise, we implement the (iterative) HITS algorithm to calculate the authority and hub scores for this graph.


### Goal:
1. Implementing HITS algorithm.
2. Identifying the best authority and hub nodes for a small graph (part `a`) as well as the `ca-GrQc.txt` dataset (part `b`)


### a)

Let the adjacency matrix for a graph of four vertices ($n_1$ to $n_4$) be
as follows:

$
A =
  \begin{bmatrix}
	0 & 1 & 1 & 1  \\
	0 & 0 & 1 & 1 \\
	1 & 0 & 0 & 1 \\
	0 & 0 & 0 & 1 \\
  \end{bmatrix}
$

Calculate the authority and hub scores for this graph using the
HITS algorithm with k = 6, and identify the best authority and
hub nodes. (A non-zero element $A_{ij}$ indicates an edge from $i$ to $j$.)

### b)
Apply the HITS algorithm to the `ca-GrQc.txt` dataset (the same dataset as for the `Exercise 2`)

**Hint:** We follow the slide "HITS algorithm" in the lecture. **Denote $x$ as authority vector and $y$ as hub vector**. You can use matrix multiplication for the update steps in the slide "Convergence of HITS". Note that rows of adjacency matrix is from-vertex and columns of adjacency matrix is to-vertex.

In [11]:
def hits_iterative(A, k = 10):
    N = A.shape[0]
    x0, y0 = 1 / (N**0.5) * np.ones(N), 1 / (N**0.5) * np.ones(N)
    xprev, yprev = x0, y0
    delta1 = delta2 = 1
    epsilon = 0.001 # We can strictly check for convergence rate of HITS algorithm
    l = 0
    while l < k and delta1 > epsilon and delta2 > epsilon:
        y = np.matmul(A, xprev)
        x = np.matmul(np.transpose(A), y) 
        x = x / np.linalg.norm(x,2)
        y = y / np.linalg.norm(y,2)
        delta1 = np.linalg.norm(x-xprev,1)
        delta2 = np.linalg.norm(y-yprev,1)
        xprev = x
        yprev = y
        l += 1
    
    print("Ran a total of {0} iterations with the convergence rate delta1, delta2={1},{2}".format(l, delta1, delta2))
    return xprev, yprev

### Part `a` dataset

In [12]:
A=np.array([
    [0, 1, 1, 1], 
    [0, 0, 1, 1], 
    [1, 0, 0, 1],
    [0, 0, 0, 1],
])
x, y = hits_iterative(A, k=6)
print("Result using iterative method:\n Authoriy vector x={0}\n Hub vector y={1}".format(x, y))

Ran a total of 4 iterations with the convergence rate delta1, delta2=0.0006129616443491248,0.0016852780559585279
Result using iterative method:
 Authoriy vector x=[0.16854345 0.27253834 0.49794598 0.80583234]
 Hub vector y=[0.65541849 0.54207544 0.40532459 0.33510118]


### part `b` dataset
You can reuse the link matrix $L$ (from Exercise 2) to compute the adjacency matrix $A$ of the dataset

In [17]:
A = np.transpose(L) # your code here
print(A)

[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]


Run HITS algorithm:

In [18]:
x, y = hits_iterative(A, 100)
print("Result using iterative method:\n Authoriy vector x={0}\n Hub vector y={1}".format(x, y))

Ran a total of 22 iterations with the convergence rate delta1, delta2=0.0008757061781108388,0.0010491474133433752
Result using iterative method:
 Authoriy vector x=[3.48948782e-05 1.45965227e-05 6.68131191e-06 ... 2.07766184e-61
 2.07766184e-61 2.07766184e-61]
 Hub vector y=[3.48948790e-05 1.45965251e-05 6.68131268e-06 ... 4.73879981e-60
 4.73879981e-60 4.73879981e-60]
